<a href="https://www.kaggle.com/code/sureshvj/text-classifier-with-lstm?scriptVersionId=188128953" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 1. Getting Datasset Link

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sms-spam-collection-dataset/spam.csv


# 2. Reading Data
  > #### Lets read the data with pandas ***read_csv()*** function.
  > #### Data Link: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset
  

In [2]:
import pandas as pd

# Reading data.
data = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv',
                  encoding='latin-1')

# 3. Analysing data
## 3.1. Data Observation:

In [3]:
# dispaly data
display(data.head())

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


## 3.2. Understanding Statistical Information of Data:

In [4]:
print(data['v1'].unique())
# Info() will give all statistical information about data.
data.info()

['ham' 'spam']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


> #### Let's understand what data is available in the last three columns.

In [5]:
f_data = data.dropna(subset = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 0)
f_data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
281,ham,\Wen u miss someone,the person is definitely special for u..... B...,why to miss them,"just Keep-in-touch\"" gdeve.."""
1038,ham,"Edison has rightly said, \A fool can ask more ...",GN,GE,"GNT:-)"""
2255,ham,I just lov this line: \Hurt me with the truth,I don't mind,i wil tolerat.bcs ur my someone..... But,"Never comfort me with a lie\"" gud ni8 and swe..."
3525,ham,\HEY BABE! FAR 2 SPUN-OUT 2 SPK AT DA MO... DE...,HAD A COOL NYTHO,TX 4 FONIN HON,"CALL 2MWEN IM BK FRMCLOUD 9! J X\"""""
4668,ham,"When I was born, GOD said, \Oh No! Another IDI...",GOD said,"\""OH No! COMPETITION\"". Who knew","one day these two will become FREINDS FOREVER!"""
5048,ham,"Edison has rightly said, \A fool can ask more ...",GN,GE,"GNT:-)"""


## 3.3. Data Analysis Impression:
> 1. Based on the above analysys **we need** only the first two columns **v1 and v2**. The **last 3 columns** we can **delete**.
> 2. **Renaming and typecasting** for the **first two** columns are required.

# 4. Data Preprocessing
## Preprocessing Steps:
> #### 1. Delete unwanted columns.
> #### 2. Rename the required columns.
> #### 3. Typescast the required columns.
> #### 4. Apply label encoding on target column.

## 4.1. Delete Unwanted Columns:
>#### **Deleting columns** which are having **more than 50 null values**.

In [6]:
# Checking columns which are having more than 50 null values.
null_columns_info = data.isna().sum()>50 # dtype: dict

# Taking column names which are having more than 50 null values.
null_columns = list(null_columns_info[null_columns_info.values == True].keys()) # dtype : list

data.drop(null_columns, axis = 1, inplace= True) # Deleting unwanted columns.
data.head() # Final data after droup the columns.

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## 4.2. Rename Required Columns:
> #### Using function: ***df.rename({old_column_name : new_column_name, ...})***
> #### Let's rename the column **v1 as 'span_or_ham'** and **v2 as 'text_context'**.

In [7]:
data.rename({'v1' : 'span_or_ham', 'v2' : "text_context"}, inplace=True, axis=1)
print(data.dtypes)
data.head()

span_or_ham     object
text_context    object
dtype: object


,span_or_ham,text_context
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## 4.3. Typescast the Required Columns:
> #### Using function: ***df.astype({column_name : dtype, ...})***
> #### Let's **change span_or_ham**, and **text_context** columns as **string**.

In [8]:
data = data.astype({'span_or_ham' : str, "text_context" : str})
data.dtypes

span_or_ham     object
text_context    object
dtype: object

## 4.4. Apply Label Encoding on Target Column

In [9]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
data['span_or_ham'] = lb.fit_transform(data['span_or_ham'])
data.head()

,span_or_ham,text_context
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


## 4.5. Cleaning Text Data:
> #### 1. Removing stop words
> #### 2. Removing special characters.
> #### 3. Converting text into lowercase.


In [10]:
import spacy as sp

nlp = sp.load('en_core_web_sm')
def clean_text(text: str) -> str:
    doc = nlp(text)
    return ' '.join([token.text.lower() for token in doc if (token.is_alpha or token.is_digit or token.is_space) and not token.is_stop])

data['text_context'] = data['text_context'].apply(clean_text)
data.head()

,span_or_ham,text_context
0,0,jurong point crazy available bugis n great wor...
1,0,ok lar joking wif u oni
2,1,free entry 2 wkly comp win fa cup final tkts 2...
3,0,u dun early hor u c
4,0,nah think goes usf lives


# 5. Training LSTM:

In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

2024-07-13 16:51:05.736483: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 16:51:05.736615: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 16:51:05.905424: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## 5.1. Preprocess the Text Data

In [12]:
# Preprocess the text data
max_words = 5000
max_len = 100

tokenizer = Tokenizer(num_words=max_words, lower=True)
tokenizer.fit_on_texts(data['text_context'].values)
sequences = tokenizer.texts_to_sequences(data['text_context'].values)
X = pad_sequences(sequences, maxlen=max_len)

## 5.2. Split the Data Into Training and Test Sets:

In [13]:
# Split the data into training and test sets
Y = data['span_or_ham'].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

## 5.3. Build the LSTM Model:

In [14]:
# Build the LSTM model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


## 5.4. Train the Model:

In [15]:
# Train the model
batch_size = 64
epochs = 5

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test), verbose=2)

Epoch 1/5
70/70 - 12s - 170ms/step - accuracy: 0.9091 - loss: 0.2667 - val_accuracy: 0.9686 - val_loss: 0.1338
Epoch 2/5
70/70 - 8s - 119ms/step - accuracy: 0.9814 - loss: 0.0711 - val_accuracy: 0.9803 - val_loss: 0.0706
Epoch 3/5
70/70 - 9s - 122ms/step - accuracy: 0.9897 - loss: 0.0344 - val_accuracy: 0.9812 - val_loss: 0.0636
Epoch 4/5
70/70 - 9s - 122ms/step - accuracy: 0.9948 - loss: 0.0181 - val_accuracy: 0.9803 - val_loss: 0.0651
Epoch 5/5
70/70 - 9s - 125ms/step - accuracy: 0.9964 - loss: 0.0133 - val_accuracy: 0.9794 - val_loss: 0.0663


## 5.5. Evaluate the Model:

In [16]:
# Evaluate the model
score = model.evaluate(X_test, Y_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.06633779406547546 / Test accuracy: 0.9793722033500671


## 5.6. Save the Model:

In [17]:
# Save the model
model.save('spam_classifier.h5')

## 5.7. Prediction:

In [18]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the trained model
model = load_model('spam_classifier.h5')

# Function to preprocess input text
def preprocess_text(text, tokenizer, max_len):
    sequences = tokenizer.texts_to_sequences([text])
    padded_sequences = pad_sequences(sequences, maxlen=max_len)
    return padded_sequences

# Sample text for prediction
sample_text = ["Congratulations! You've won a $1,000 Walmart gift card. Go to http://bit.ly/12345 to claim now."]

# Preprocess the sample text
preprocessed_text = preprocess_text(sample_text[0], tokenizer, max_len)

# Make prediction
prediction = model.predict(preprocessed_text)
prediction_label = (prediction > 0.5).astype(int)  # Binarize the output

# Decode the label
label = 'spam' if prediction_label[0][0] == 1 else 'ham'
print(f'Text: "{sample_text[0]}" is classified as {label}.')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
Text: "Congratulations! You've won a $1,000 Walmart gift card. Go to http://bit.ly/12345 to claim now." is classified as spam.


Refs:
    
    https://www.kaggle.com/code/amvillalobos/rnn-text-classification
    https://www.kaggle.com/code/fareselmenshawii/rnn-from-scratch
    https://www.kaggle.com/code/kcsener/8-recurrent-neural-network-rnn-tutorial
    https://www.kaggle.com/code/thebrownviking20/intro-to-recurrent-neural-networks-lstm-gru
    https://www.datacamp.com/tutorial/tutorial-for-recurrent-neural-network
    https://www.kaggle.com/code/matleonard/text-classification/notebook
    https://www.kaggle.com/code/ngawangchoeda/spam-classifier-using-lstm
            
    